In [8]:
import sys
import time
sys.path.append("../Common/Crawler")
sys.path.append("../Common/Util")

from mod_crawler_base import crawler_engine as crawler_main
from time import sleep
from selenium import webdriver

szKeyword    = "키워드로 보는 사설"
szKeyword_En = "Keywordsasul"

In [18]:
#book cosmos용 entity checker이다.
class crawler_joins_kwd (crawler_main):
    def __init__( self , isLinux, isHidden, keyword_en ) :
        super().__init__(isHidden, ".", keyword_en)
        
    def setKeyword( self, keyword ):
        self.keyword = keyword
        
    def run(self):
        self.fetch();
        
        self.navigate(baseUrl)
        sleep(1)
    
    def getMaxPages_(self):
        baseUrl = "https://news.joins.com/find/list?page=1&IsDuplicate=True&key=EditorialColumn&Keyword=%s&SourceGroupType=Joongang" % (self.keyword)
        css_name = "btn_next"
        self.openPage(baseUrl)
        while (True):
            elem = self.webDrv.find_element_by_class_name(css_name)
            pos = elem.text.find("없음")
            if (pos == -1):
                print("다음페이지 있음")
                elem.click()
            else:
                print("다음페이지 없음")
                elems = self.webDrv.find_elements_by_class_name('link_page')
                listPages = []
                for eobj in elems:
                    listPages.append(int(eobj.text))
                listPages = sorted(listPages, reverse=True)
                return listPages[0]
            sleep(0.5)
    
    def fetch(self):
                
        max_page = self.getMaxPages_()
        for i in range(1, max_page + 1):            
            now = time.localtime()            
            url = "https://news.joins.com/find/list?page=%d&IsDuplicate=True&key=EditorialColumn&Keyword=%s&SourceGroupType=Joongang" % (i, self.keyword )
            self.openPage( url )
            listLink = []
            listElem = self.webDrv.find_elements_by_class_name('headline')
            if (self.isTxt == False ) :
                print("text file isnt configured.")
                return
            for i, elem in enumerate(listElem):
                listLink.append(elem.find_element_by_tag_name('a').get_attribute('href'))
            for i, elem in enumerate(listLink):
                url = listLink[i]
                if( self.logger.getHistory(url) == False ) :
                    try:
                        start_time = time.time()
                        self.openPage(url)
                        self.crawContents(True)
                        self.logger.updateHistory(url, "ok")
                    except:
                        self.logger.updateHistory(url, "fail")
                sleep(1)
            self.logger.close()            
            sleep(2)
        
    def scrape(self,isShowContent):
        #article_title
        elem = self.webDrv.find_element_by_id('article_title')
        txt_head = elem.text
        self.txt.write(txt_head)
        #byline
        elem = self.webDrv.find_element_by_class_name('byline')
        txt_date_input = elem.text.split()[2]
        self.txt.write(txt_date_input)
        #profile
        elem = self.webDrv.find_element_by_class_name('profile')
        txt_profile = elem.text
        self.txt.write(txt_profile)
        # tags , |로 구분
        elem = self.webDrv.find_element_by_class_name('tag_list')
        listTags = elem.text.split("#")[1:]
        szTags = ""
        for tags in listTags:
            szTags += tags.replace("\n", "")
            szTags += "|"
        if (isShowContent): print(szTags)
        self.txt.write(szTags)
        #article_body
        elem = self.webDrv.find_element_by_class_name('article_body')
        txt_org = elem.text
        txt_proc = txt_org.replace("\n", "  ")
        if (isShowContent): print(txt_proc)
        self.txt.writeLast(txt_proc)


In [19]:
MOD = crawler_joins_kwd( False, False, "joins_keywordNSasul"  )
MOD.setKeyword( "키워드로 보는 사설" )
        


[crawler_base] platform is identified with linux 
[Crawler_Logger] create new file
[Crawler_Logger] history node: None
[crawler_base] path of chrome driver is /root/Python/couragesuper-ds/Common/Crawler/lib/chromedriver 
[crawler_base] chromdriver option is hidden


In [20]:
MOD.run()

[crawler_base] openPage = https://news.joins.com/find/list?page=1&IsDuplicate=True&key=EditorialColumn&Keyword=키워드로 보는 사설&SourceGroupType=Joongang
다음페이지 있음
다음페이지 없음
[crawler_base] openPage = https://news.joins.com/find/list?page=1&IsDuplicate=True&key=EditorialColumn&Keyword=키워드로 보는 사설&SourceGroupType=Joongang
[craw_history_logger] url=https://news.joins.com/article/22846477
[crawler_base] openPage = https://news.joins.com/article/22846477
[Crawler_Logger] history node: {'url': 'https://news.joins.com/article/22846477'}
[craw_history_logger] url=https://news.joins.com/article/22827287
[crawler_base] openPage = https://news.joins.com/article/22827287
[Crawler_Logger] history node: {'url': 'https://news.joins.com/article/22827287'}
[craw_history_logger] url=https://news.joins.com/article/22807018
[crawler_base] openPage = https://news.joins.com/article/22807018
[crawler][base] with URL 
[Crawler_Logger] history node: {'url': 'https://news.joins.com/article/22807018'}
[craw_history_logger

KeyboardInterrupt: 

In [21]:
MOD.close()